# Projet programmation concurrente :

In [ ]:
from threading import Thread
from random import randint
import time

class Producteur(Thread) :

    NB_MIN = 1
    NB_MAX = 100

    def __init__(self, queue, temps, label, num_producteur) :
        Thread.__init__(self)
        self.queue = queue
        self.n = randint(self.NB_MIN,self.NB_MAX)
        self.temps = temps
        self.label = label
        self.num_producteur = num_producteur
        self.interrupted = False

    def setInterrupted(self,bool) :
        self.interrupted = bool

    def run(self) :
        while not self.interrupted :
            self.label.config(text="Producteur"+str(self.num_producteur)+" : Ajout de l'entier "+str(self.n)+"...")
            self.queue.enfiler(self.n)
            self.label.config(text="Producteur"+str(self.num_producteur)+" : Ajout de l'entier "+str(self.n)+" effectue.")
            self.n = randint(self.NB_MIN,self.NB_MAX)
            time.sleep(self.temps/1000)

In [ ]:
from threading import Thread
import time

class Consommateur(Thread) :

    def __init__(self, queue, temps, label, num_consommateur) :
        Thread.__init__(self)
        self.queue = queue
        self.temps = temps
        self.label = label
        self.num_consommateur = num_consommateur
        self.interrupted = False

    def setInterrupted(self,bool) :
        self.interrupted = bool

    def run(self) :
        while not self.interrupted :
            self.label.config(text="Consommateur"+str(self.num_consommateur)+" : Retrait d'un entier en cours...")
            n = self.queue.defiler()
            self.label.config(text="Consommateur"+str(self.num_consommateur)+" : retrait de l'entier "+str(n)+" effectue.")
            time.sleep(self.temps/1000)


In [ ]:
from threading import Condition

class Queue() :

    def __init__(self, capacite, l_liste, l_taille) :
        self.valeurs = []
        self.capacite = capacite
        self.l_liste = l_liste
        self.l_taille = l_taille
        self.verrou = Condition()
        self.l_liste.config(text="File = "+self.createString())


    def majLabel(self) :
        self.l_liste.config(text="File = "+self.createString())
        self.l_liste.update()
        self.l_taille.update()

    def defiler(self) :
        with self.verrou :
            while len(self.valeurs) == 0 : self.verrou.wait()
            n = self.valeurs[0]
            del self.valeurs[0]
            self.majLabel()
            self.verrou.notifyAll()
            return n

    def enfiler(self, n) :
        with self.verrou :
            while len(self.valeurs) == self.capacite : self.verrou.wait()
            self.valeurs.append(n)
            self.majLabel()
            self.verrou.notifyAll()

    def createString(self) :
        with self.verrou :
            s = "[ "
            for i in range(0,len(self.valeurs)) :
                s+=str(self.valeurs[i])
                if i != len(self.valeurs)-1 :  s+=", "
            s+=" ]"
            self.l_taille.config(text="Longueur de la file : "+ str(len(self.valeurs)) );
            return s;

In [ ]:
from tkinter import *
from random import randint

class Fenetre(Frame) :
    MIN_TEMPS_PROD = 2000
    MAX_TEMPS_PROD = 2500
    MIN_TEMPS_CONSO = 1700
    MAX_TEMPS_CONSO = 1900
    TAILLE_MAX_FILE = 20

    def __init__(self, fenetre, nb_producteurs, nb_consommateurs) :
        Frame.__init__(self)
        self.pack(fill=BOTH)
        self.nb_consommateurs = nb_consommateurs
        self.nb_producteurs = nb_producteurs
        self.liste_prod = []
        self.liste_cons = []

        self.f_liste = Frame(self)
        self.f_liste.pack(side=TOP, fill=BOTH)

        self.l_liste = Label(self.f_liste, text="< <", fg="purple")
        self.l_liste.pack(side=LEFT, fill=BOTH)

        self.l_taille = Label(self.f_liste, text="taille de la liste : 0", fg="gray")
        self.l_taille.pack(side=RIGHT, fill=BOTH)

        self.b_start = Button(self, text="Start", command=self.cliquer)
        self.b_start.pack(side=BOTTOM, fill=BOTH)

        self.p_threads = PanedWindow(self, orient=VERTICAL)
        self.p_threads.pack(side=BOTTOM, fill=BOTH)

        self.file = Queue(self.TAILLE_MAX_FILE, self.l_liste, self.l_taille)

    def create_process(self) :
        for i in range(self.nb_producteurs) :
            label_p = Label(self.p_threads, text="Consommateur"+str(i), fg="black")
            t_p = randint(self.MIN_TEMPS_PROD,self.MAX_TEMPS_PROD)
            self.liste_prod.append(Producteur(self.file, t_p, label_p, i))
            self.p_threads.add(label_p)

        for i in range(self.nb_consommateurs) :
            label_c = Label(self.p_threads, text="Producteur"+str(i), fg="blue")
            t_c = randint(self.MIN_TEMPS_CONSO,self.MAX_TEMPS_CONSO)
            self.liste_cons.append(Consommateur(self.file, t_c, label_c, i))
            self.p_threads.add(label_c)

    def start_process(self) :
        for i in range(self.nb_producteurs) : self.liste_prod[i].start()
        for i in range(self.nb_consommateurs) : self.liste_cons[i].start()

    def stop_process(self) :
        for i in range(self.nb_producteurs) : self.liste_prod[i].setInterrupted(True)
        for i in range(self.nb_consommateurs) : self.liste_cons[i].setInterrupted(True)



    def cliquer(self) :
        if self.b_start.cget("text") == "Start" :
            self.create_process()
            self.start_process()
            self.b_start.config(text="Stop")
        elif self.b_start.cget("text") == "Stop" :
            self.stop_process()
            self.b_start.config(state=DISABLED)

if __name__ == '__main__':
    fenetre = Tk()
    fenetre.geometry('650x200')
    Fenetre = Fenetre(fenetre, 4,2)
    Fenetre.mainloop()